In [7]:
from pydantic import BaseModel, Field
from instructor import OpenAISchema
from typing import List

class Trigger(OpenAISchema):
    def execute(self):
        raise NotImplementedError("This method should be overridden by subclasses.")

class Action(OpenAISchema):
    def execute(self):
        raise NotImplementedError("This method should be overridden by subclasses.")

class CreateIssue(Action):
    """
    Create a new issue in a repository.
    """
    owner: str = Field(..., description="Owner of the repository", examples=["openai", "facebook"])
    repo: str = Field(..., description="Name of the repository", examples=["gpt-3", "react"])
    title: str = Field(..., description="Title of the issue", examples=["Bug in the code", "Feature request"])
    body: str = Field(default="", description="Body of the issue", examples=["The code is not working", "I would like to request a new feature"])

    def execute(self):
        return f"Creating issue '{self.title}' in '{self.owner}/{self.repo}'"

class Tool():
    def __init__(self, name: str = None, description: str = None, actions: List[Action] = [], triggers: List[Trigger] = []):
        self.name = name
        self.description = description
        self.actions = actions
        self.triggers = triggers

    def execute_action(self, action_name: str, request_data: dict):
        if action_name in self.actions:
            return self.actions[action_name](**request_data).execute()
        else:
            return "Action not found"
        
    def json_schema(self):
        return {
            "Name": self.name,
            "Description": self.description,
            "Actions": [
                action.openai_schema for action in self.actions
            ],
            "Triggers": [
                trigger.openai_schema for trigger in self.triggers
            ],
        }

Github = Tool(
    name="Github",
    description="Github API",
    actions=[CreateIssue],
    triggers=[]
)

Github.json_schema()

{'Name': 'Github',
 'Description': 'Github API',
 'Actions': [{'name': 'CreateIssue',
   'description': 'Create a new issue in a repository.',
   'parameters': {'properties': {'owner': {'description': 'Owner of the repository',
      'examples': ['openai', 'facebook'],
      'title': 'Owner',
      'type': 'string'},
     'repo': {'description': 'Name of the repository',
      'examples': ['gpt-3', 'react'],
      'title': 'Repo',
      'type': 'string'},
     'title': {'description': 'Title of the issue',
      'examples': ['Bug in the code', 'Feature request'],
      'title': 'Title',
      'type': 'string'},
     'body': {'default': '',
      'description': 'Body of the issue',
      'examples': ['The code is not working',
       'I would like to request a new feature'],
      'title': 'Body',
      'type': 'string'}},
    'required': ['owner', 'repo', 'title'],
    'type': 'object'}}],
 'Triggers': []}